In [1]:
import sys
import torch
from torch.utils.data import DataLoader, Subset
from dotenv import load_dotenv
import os
from pathlib import Path

load_dotenv() 

# Check device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Load paths from environment variables
PARENT_PATH = Path(os.getenv("PARENT_DIR", "."))  # default fallback to current directory
print(f"Parent directory: {PARENT_PATH}")
data_root = PARENT_PATH / "data"
print(data_root)
# data_root = Path(os.getenv("DATA_ROOT", "./data"))  # default fallback to ./data
image_data_path = "audio/for-norm/for-norm/"
data_root = data_root/ image_data_path

print(f"Data directory: {data_root}")
print(f"Exists: {data_root.exists()}")


# Add src to path
src_path = Path(PARENT_PATH / "src")
sys.path.insert(0, str(src_path))



Using device: cuda
Parent directory: D:\projects\RealTimeFakeCheck
D:\projects\RealTimeFakeCheck\data
Data directory: D:\projects\RealTimeFakeCheck\data\audio\for-norm\for-norm
Exists: False


In [2]:
from loaders.audio_loader import AudioDataLoader
from encoders.audio_encoder import AudioEmbeddingModel

In [4]:
loader = AudioDataLoader(data_root)
train_loader = loader.get_dataloader("train")

Resolving data files:   0%|          | 0/53868 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/10798 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/4634 [00:00<?, ?it/s]

Map:   0%|          | 0/53868 [00:00<?, ? examples/s]

C:\Users\muthu\miniconda3\envs\multi_env\Lib\site-packages\transformers\models\wav2vec2\feature_extraction_wav2vec2.py:92: RuntimeWarning: Mean of empty slice.
  normed_slice = (vector - vector[:length].mean()) / np.sqrt(vector[:length].var() + 1e-7)
C:\Users\muthu\miniconda3\envs\multi_env\Lib\site-packages\numpy\_core\_methods.py:145: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\muthu\miniconda3\envs\multi_env\Lib\site-packages\transformers\models\wav2vec2\feature_extraction_wav2vec2.py:92: RuntimeWarning: Degrees of freedom <= 0 for slice
  normed_slice = (vector - vector[:length].mean()) / np.sqrt(vector[:length].var() + 1e-7)
C:\Users\muthu\miniconda3\envs\multi_env\Lib\site-packages\numpy\_core\_methods.py:181: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
C:\Users\muthu\miniconda3\envs\multi_env\Lib\site-packages\numpy\_core\_methods.py:215: RuntimeWarning: invalid va

Map:   0%|          | 0/10798 [00:00<?, ? examples/s]

In [7]:
train_loader

In [8]:
model = AudioEmbeddingModel().to(device)
model.eval()

all_embeddings = []
all_labels = []

with torch.no_grad():
    for index,batch in enumerate(train_loader):
        print(f"  Processing batch {index+1}/{len(train_loader)}...")
        input_values = batch["input_values"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        emb = model(input_values, attention_mask)

        all_embeddings.append(emb.cpu())
        all_labels.append(batch["labels"])

all_embeddings = torch.cat(all_embeddings)
all_labels = torch.cat(all_labels)

KeyError: "Invalid key: 0. Please first select a split. For example: `my_dataset_dictionary['train'][0]`. Available splits: ['train', 'validation']"

In [ ]:
save_path = "embeddings/audio_embeddings.pt"
torch.save({"embeddings": all_embeddings, "labels": all_labels}, save_path)

print("Saved:", save_path)

In [ ]:
data = torch.load("embeddings/audio_embeddings.pt")
audio_embeddings = data["embeddings"]    # torch.Tensor
audio_labels = data["labels"]            # torch.Tensor
audio_embeddings, audio_labels